In [1]:
import geopandas as gpd
import pandas as pd
import shapely
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
capmerge = pd.read_csv('../data/capmerge.csv')
georegions = pd.read_csv('../data/georegions.csv')
ratesbymetro = pd.read_csv('../data/ratesbymetro.csv')

In [3]:
capmerge['install_ratio'] = round((capmerge.existing_installs_count / capmerge.count_qualified) * 100,2)
ratesbymetro['avg_rate'] = ratesbymetro['Thousand Dollars'] / ratesbymetro['Megawatthours']
ratesbymetro['usage'] = ratesbymetro['Megawatthours'] / ratesbymetro['Count']

In [4]:
dataset = capmerge.merge(georegions.merge(ratesbymetro, how='inner', on='GEOID10'), how='inner', on='region_name')

In [5]:
dataset[dataset.state_name_x == 'New Mexico']

,region_name,state_name_x,lat_max,lat_min,lng_max,lng_min,lat_avg,lng_avg,yearly_sunlight_kwh_kw_threshold_avg,count_qualified,...,Utility Name,Part,Service Type,Ownership,Thousand Dollars,Megawatthours,Count,eiaid,avg_rate,usage
146,35049010602,New Mexico,35.559280,35.494301,-105.920258,-105.983429,35.531593,-105.955711,1409.30,41,...,Mora-San Miguel Elec Coop,A,Bundled,Cooperative,10330.0,60027.0,10822.0,12901.0,0.172089,5.546757
278,35043010721,New Mexico,35.306339,35.280140,-106.696960,-106.716904,35.294823,-106.706734,1417.80,96,...,Public Service Co of NM,A,Bundled,Investor Owned,427882.7,3227338.0,471935.0,15473.0,0.132581,6.838522
7809,35061970303,New Mexico,34.804482,34.652851,-106.673599,-106.750977,34.745792,-106.718697,1418.65,431,...,Public Service Co of NM,A,Bundled,Investor Owned,427882.7,3227338.0,471935.0,15473.0,0.132581,6.838522
8086,35049001301,New Mexico,35.635681,35.623291,-106.045097,-106.071220,35.630455,-106.058380,1409.30,542,...,Public Service Co of NM,A,Bundled,Investor Owned,427882.7,3227338.0,471935.0,15473.0,0.132581,6.838522
8883,35001003731,New Mexico,35.195202,35.174110,-106.496140,-106.533447,35.183872,-106.515457,1421.20,238,...,Public Service Co of NM,A,Bundled,Investor Owned,427882.7,3227338.0,471935.0,15473.0,0.132581,6.838522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44028,35013000201,New Mexico,32.313091,32.277199,-106.774788,-106.815590,32.295971,-106.795990,1416.95,2131,...,El Paso Electric Co,A,Bundled,Investor Owned,75401.0,720397.0,88405.0,5701.0,0.104666,8.148826
44371,35043010703,New Mexico,35.297192,35.256809,-106.605980,-106.686981,35.275475,-106.645508,1417.80,3469,...,Public Service Co of NM,A,Bundled,Investor Owned,427882.7,3227338.0,471935.0,15473.0,0.132581,6.838522
44459,35001001200,New Mexico,35.078178,35.023449,-106.610451,-106.639938,35.052589,-106.625603,1421.20,2131,...,Public Service Co of NM,A,Bundled,Investor Owned,427882.7,3227338.0,471935.0,15473.0,0.132581,6.838522
44926,35001003400,New Mexico,35.137451,35.105160,-106.604073,-106.639229,35.121647,-106.619499,1421.20,1731,...,Public Service Co of NM,A,Bundled,Investor Owned,427882.7,3227338.0,471935.0,15473.0,0.132581,6.838522


In [6]:
dataset = dataset.astype({'region_name':'str'})
dataset['region_name'] = dataset['region_name'].str.zfill(11)
dataset[dataset.state_name_x == 'New Mexico']['region_name']

146      35049010602
278      35043010721
7809     35061970303
8086     35049001301
8883     35001003731
            ...     
44028    35013000201
44371    35043010703
44459    35001001200
44926    35001003400
44989    35001003736
Name: region_name, Length: 214, dtype: object

In [7]:
dataset['savings_10kw_house'] = (((10 * dataset['yearly_sunlight_kwh_kw_threshold_avg']) * dataset['avg_rate']) / .85) - (dataset['usage'] * dataset['avg_rate'])

In [8]:
dataset['state_name_x'].sort_values().unique()

array(['Aguadilla', 'Alabama', 'Alaska', 'Arecibo', 'Arizona', 'Arkansas',
       'Bayamón', 'Cabo Rojo', 'California', 'Canóvanas', 'Carolina',
       'Cataño', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Dorado', 'Florida', 'Georgia', 'Guaynabo',
       'Hatillo', 'Hawaii', 'Hormigueros', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Mayagüez', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Ponce', 'Rhode Island', 'San Juan',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Toa Alta',
       'Toa Baja', 'Trujillo Alto', 'Utah', 'Vermont', 'Virginia',
       'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [9]:
#nmtracts = nmtracts.set_crs('epsg:3857')

In [10]:
nmtracts = gpd.read_file('../data/NM/tl_2018_35_tract.json')
nmtracts = nmtracts[nmtracts['GEOID'].isin(dataset[dataset.state_name_x == 'New Mexico']['region_name'])]
center = nmtracts[nmtracts.GEOID == '35001003400'].geometry.centroid
area_center = [center.y, center.x]
print(area_center)

[130    35.121525
dtype: float64, 130   -106.619541
dtype: float64]


<ipython-input-10-9d6d7c389da0>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = nmtracts[nmtracts.GEOID == '35001003400'].geometry.centroid


In [11]:
nmmap = folium.Map(location = area_center, zoom_start = 12)

marker_cluster = MarkerCluster().add_to(nmmap)

folium.GeoJson(data= nmtracts['geometry']).add_to(nmmap)

folium.Choropleth(
    geo_data=nmtracts,
    data=dataset[dataset['state_name_x']=='New Mexico'],
    columns=['region_name','install_ratio'],
    key_on='feature.properties.GEOID',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.5,
    bins=8,
    legend_name='Install Ratio'
             ).add_to(nmmap)

for row_i, row_val in dataset[dataset['state_name_x']=='New Mexico'].iterrows():
    #center = row_val.shapes.geometry.centroid
    loc = [(row_val.lat_max + row_val.lat_min)/2,(row_val.lng_max + row_val.lng_min)/2]
    pop = str(row_val['state_name_x']) + str(" - ") + str('Average Saving for a Large House: $') + str(round(row_val['savings_10kw_house'])) + str(' - Install Ratio: ') + str(row_val['install_ratio'])+'%'
    if row_val['lat_max'] > 0:
        icon = folium.Icon(icon='sun-o', prefix='fa', color='green')
    else:
        icon = folium.Icon(icon='solar-panel', prefix='fa', color='red')
    
    marker = folium.Marker(
        location = loc,
        popup = pop,
        icon = icon)
    marker.add_to(marker_cluster)
    
nmmap.save('../maps/nmmap.html')

In [12]:
catracts = gpd.read_file('../data/CA/tl_2018_06_tract.json')
catracts = catracts[catracts['GEOID'].isin(dataset[dataset.state_name_x == 'California']['region_name'])]
area_center = [32.71382390041209, -117.16951267956273]

In [13]:
camap = folium.Map(location = area_center, zoom_start = 12)

marker_cluster = MarkerCluster().add_to(camap)

folium.GeoJson(data= catracts['geometry']).add_to(camap)

folium.Choropleth(
    geo_data=catracts,
    data=dataset[dataset['state_name_x']=='California'],
    columns=['region_name','install_ratio'],
    key_on='feature.properties.GEOID',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.5,
    bins=8,
    legend_name='Install Ratio'
             ).add_to(camap)

for row_i, row_val in dataset[(dataset['state_name_x']=='California')&(dataset['savings_10kw_house']>0)].iterrows():
    #center = row_val.shapes.geometry.centroid
    loc = [(row_val.lat_max + row_val.lat_min)/2,(row_val.lng_max + row_val.lng_min)/2]
    pop = str(row_val['state_name_x']) + str(" - ") + str('Average Saving for a Large House: $') + str(round(row_val['savings_10kw_house'])) + str(' - Install Ratio: ') + str(row_val['install_ratio'])+'%' 
    if row_val['lat_max'] > 0:
        icon = folium.Icon(icon='sun-o', prefix='fa', color='green')
    else:
        icon = folium.Icon(icon='solar-panel', prefix='fa', color='red')
    
    marker = folium.Marker(
        location = loc,
        popup = pop,
        icon = icon)
    marker.add_to(marker_cluster)
    
camap.save('../maps/camap.html')                                             

In [14]:
hitracts = gpd.read_file('../data/HI/tl_2018_15_tract.json')
hitracts = hitracts[hitracts['GEOID'].isin(dataset[dataset.state_name_x == 'Hawaii']['region_name'])]
area_center = [21.4, -157.9]

In [15]:
himap = folium.Map(location = area_center, zoom_start = 12)

marker_cluster = MarkerCluster().add_to(himap)

folium.GeoJson(data= hitracts['geometry']).add_to(himap)

folium.Choropleth(
    geo_data=hitracts,
    data=dataset[dataset['state_name_x']=='Hawaii'],
    columns=['region_name','install_ratio'],
    key_on='feature.properties.GEOID',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.5,
    bins=8,
    legend_name='Install Ratio'
             ).add_to(himap)

for row_i, row_val in dataset[(dataset['state_name_x']=='Hawaii')&(dataset['savings_10kw_house']>0)].iterrows():
    #center = row_val.shapes.geometry.centroid
    loc = [(row_val.lat_max + row_val.lat_min)/2,(row_val.lng_max + row_val.lng_min)/2]
    pop = str(row_val['state_name_x']) + str(" - ") + str('Average Saving for a Large House: $') + str(round(row_val['savings_10kw_house'])) + str(' - Install Ratio: ') + str(row_val['install_ratio'])+'%' 
    if row_val['lat_max'] > 0:
        icon = folium.Icon(icon='sun-o', prefix='fa', color='green')
    else:
        icon = folium.Icon(icon='solar-panel', prefix='fa', color='red')
    
    marker = folium.Marker(
        location = loc,
        popup = pop,
        icon = icon)
    marker.add_to(marker_cluster)
    
himap.save('../maps/himap.html')                                             